In [25]:
import pickle
import random
import copy
import numpy
import pandas as pd
alphabet = ['A', 'G', 'C', 'T']

In [26]:
def GibbsSampler(S,L):

    PWM = []
    Background = [0.25, 0.25, 0.25, 0.25] #matrix of background frequencies
    for i in range(len(alphabet)):
        PWM.append([0]*L)

    a = [] #initialize matrix of random motif starting positions
    for i in S:
        a.append(random.randint(0,len(i)-L))

    PWM = recalculatePWM(S,a,PWM,L) #initialize PWM according to random positions in a

    finish = False
    count=0
    while (finish==False):
        removeind = random.randint(0,len(S)-1)
        x = S.pop(removeind)
        oldA = a.pop(removeind) #randomly select a sequence to remove from x (and correspondingly remove entry in a)

        oldPWM = copy.deepcopy(PWM)
        PWM = recalculatePWM(S, a, PWM, L) #reculate PWM for the remaining sequences

        #generate matrix of scores for each possible starting point in sequence x
        Z_ij = []
        totalscore = 0
        for i in range(len(x)-L):
            num=1
            denom=1
            for j in range(L):
                if(x[i+j]=='A'):
                    num*=PWM[0][j]
                    denom*=Background[0]
                if (x[i + j] == 'G'):
                    num *= PWM[1][j]
                    denom *= Background[1]
                if (x[i + j] == 'C'):
                    num *= PWM[2][j]
                    denom *= Background[2]
                if (x[i + j] == 'T'):
                    num *= PWM[3][j]
                    denom *= Background[3]
            Z_ij.append(num/denom)
            totalscore += num/denom

        #normalize the entries in Z_ij by dividing by the total score so that Z_ij becomes a probability distribution for starting positions in x
        for i in range(len(Z_ij)):
            Z_ij[i] = Z_ij[i]/totalscore

        newA = (numpy.random.choice(len(x)-L, p=Z_ij)) #randomly select a new starting position for x based on normalized probabilities in Z_ij
        a.append(newA)
        S.append(x)

        #check for convergence
        change = 0
        for i in range(len(PWM)):
            for j in range(len(alphabet)):
                if(change<abs(oldPWM[j][i] - PWM[j][i])):
                    change = abs(oldPWM[j][i] - PWM[j][i])

        #Termination condition: run for 100 loops
        #if(count==100):
        if(change<0.001):
            finish = True

        count+=1

    return PWM

In [27]:
#update the PWM using the starting positions stored in a
def recalculatePWM(S, a, PWM, L):
    for i in range(L):
        Acount = 0.1 #initialize with pseudocounts
        Tcount = 0.1
        Gcount = 0.1
        Ccount = 0.1
        for j in range(len(S)):
            if (S[j][a[j] + i]) == 'A':
                Acount += 1
            if (S[j][a[j] + i]) == 'T':
                Tcount += 1
            if (S[j][a[j] + i]) == 'G':
                Gcount += 1
            if (S[j][a[j] + i]) == 'C':
                Ccount += 1

        d = (Acount+Tcount+Gcount+Ccount)

        #normalize counts by length of the sequence
        PWM[0][i] = Acount / (d)
        PWM[1][i] = Gcount / d
        PWM[2][i] = Ccount / d
        PWM[3][i] = Tcount / d

    return PWM

In [22]:
#read data
    with open("hot_seq.pickle","rb") as fp:
        hotspots = pickle.load(fp)
    S = []
    for entry in hotspots:
        S.append(entry[:, :4])

In [36]:
np.asarray(S[0])

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [23]:
#convert back from one hot encoding
sequences = []
for entry in S:
    temp = ""
    for row in entry:
        if row[0]==1:
            temp+="A"
        elif row[1]==1:
            temp+="C"
        elif row[2] ==1:
            temp+="G"
        else:
            temp+="T"
    sequences.append(temp)

In [28]:
def percent_id (seq):
    motif = "CCNCCNTNNCCNC"
    count=0.0
    for i in range(len(seq)):
        if motif[i]=="N":
            count+=1
        elif motif[i]==seq[i]:
            count+=1
    percent = count/13
    return percent 

In [29]:
def GibbsT(lengthMotif, numIterations, numSeq):  
    bases = {0:'A',1:'G',2:'C',3:'T'}
    motifs = []
    percentid = []
    for i in range(numIterations):
        P = GibbsSampler(sequences[0:numSeq],lengthMotif)
        motif = ""
        for i in range(lengthMotif):
            maxi = 0
            for j in range(4):
                if (P[maxi][i] < P[j][i]): maxi = j
            motif += bases[maxi]
        motifs.append(motif)
        percentid.append(percent_id(motif))
    return motifs, percentid

In [30]:
#length of motif, number of iterations, number of sequences from hotspot list to search over
motifs, percents = GibbsT(13,10000,24) 

degenerate motif represented in 40% hotspots: CCNCCNTNNCCNC

In [31]:
import pickle
with open("foundpercent_ids.txt", "wb") as fp: 
    pickle.dump(percents, fp)
with open("foundallmotifs.txt", "wb") as fp: 
    pickle.dump(motifs, fp)

In [32]:
keys = {}
keys["NNNNNNNNNN"] = [0,0] #number of hits, percent id

for k in range(len(motifs)):
     #store motif in dictionary
    motif = motifs[k]
    if motif in keys:
        value = keys.get(motif)
        keys[motif] = [value[0] + 1, value[1]]  # increment number of hits
    else:
        keys[motif]= [0, percents[k]]

In [7]:
import pickle
with open("allmotifs.txt", "rb") as fp:
         motifs = pickle.load(fp)
with open("percent_ids.txt", "rb") as fp:
         percents = pickle.load(fp)

In [33]:

for i in range(11):
    mostfreq = "NNNNNNNNNN"
    for j in keys:
        if(keys[j][0]>keys[mostfreq][0]):
            mostfreq = j
    print(mostfreq, keys[mostfreq])
    del(keys[mostfreq])

AAAAGGGAAAGAA [1, 0.38461538461538464]
AGGAGGGGAGGGA [1, 0.38461538461538464]
GGGAAGGAGAGAG [1, 0.38461538461538464]
AAAAAGGGAAAAG [1, 0.38461538461538464]
CAGAATCTTTCTA [1, 0.5384615384615384]
GCAAAGATGAAAA [1, 0.46153846153846156]
GAAAGAGAAAAAG [1, 0.38461538461538464]
AAGGAAGAGATGG [1, 0.38461538461538464]
CAGGGGGAAAAGG [1, 0.46153846153846156]
TGGGGGGGAGGGA [1, 0.38461538461538464]
GGAGGGAGAGAGG [1, 0.38461538461538464]
